In [1]:
"""
Notebook for streaming data from a microphone in realtime

audio is captured using pyaudio
then converted from binary data to ints using struct
then displayed using matplotlib

scipy.fftpack computes the FFT

if you don't have pyaudio, then run

>>> pip install pyaudio

note: with 2048 samples per chunk, I'm getting 20FPS
when also running the spectrum, its about 15FPS
"""

import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [2]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '-', lw=2)

# create semilogx line for spectrum
# line_fft, = ax2.semilogx(xf, np.random.rand(CHUNK), '-', lw=2)
line_fft, = ax2.plot(xf, np.random.rand(CHUNK), '-', lw=2)

# Signal range is -32k to 32k
# limiting amplitude to +/- 4k
AMPLITUDE_LIMIT = 4096

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-AMPLITUDE_LIMIT, AMPLITUDE_LIMIT)
ax1.set_xlim(0, 2 * CHUNK)
plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[-AMPLITUDE_LIMIT, 0, AMPLITUDE_LIMIT])

# format spectrum axes
ax2.set_xlim(20, RATE / 2)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)    

    data_np = np.frombuffer(data, dtype='h')
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_np)

    #remove frequencies above 20kHz
    yf[round(20000*(CHUNK/RATE)):] = 0
    

    #print the dominant frequency
    dominant_freq = np.argmax(np.abs(yf[0:CHUNK])  / (512 * CHUNK))
    print(dominant_freq * RATE / CHUNK)
    print(np.abs(yf[np.argmax(np.abs(yf[0:CHUNK])  / (512 * CHUNK))])  / (512 * CHUNK))


    #remove the dominant frequency
    yf[dominant_freq-3:dominant_freq+3] = 0

    #find the second dominant frequency
    second_dominant_freq = np.argmax(np.abs(yf[0:CHUNK])  / (512 * CHUNK))
    print(second_dominant_freq * RATE / CHUNK)
    print(np.abs(yf[np.argmax(np.abs(yf[0:CHUNK])  / (512 * CHUNK))])  / (512 * CHUNK))

    #remove the second dominant frequency
    yf[second_dominant_freq-3:second_dominant_freq+3] = 0

    #find the third dominant frequency
    third_dominant_freq = np.argmax(np.abs(yf[0:CHUNK])  / (512 * CHUNK))
    print(third_dominant_freq * RATE / CHUNK)
    print(np.abs(yf[np.argmax(np.abs(yf[0:CHUNK])  / (512 * CHUNK))])  / (512 * CHUNK))

    #remove the third dominant frequency
    yf[third_dominant_freq-3:third_dominant_freq+3] = 0
    

    line_fft.set_ydata(np.abs(yf[0:CHUNK])  / (512 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2642:(snd

stream started
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.00033092498779296875
0.0
0.00033092498779296875
0.0
0.00033092498779296875
0.0
0.004094123840332031
0.0
0.004094123840332031
0.0
0.004094123840332031
0.0
0.026079177856445312
0.0
0.026079177856445312
0.0
0.026079177856445312
21.533203125
0.0879314258930533
21.533203125
0.0879314258930533
21.533203125
0.0879314258930533
64.599609375
0.0188864644893638
129.19921875
0.01685502095342346
236.865234375
0.01319927459433394
107.666015625
0.02247787582780013
21.533203125
0.016629146472604585
21.533203125
0.016629146472604585
107.666015625
0.021517655153042554
21.533203125
0.02000864458495431
21.533203125
0.02000864458495431
64.599609375
0.04076191934648137
236.865234375
0.020848818337498184
129.19921875
0.01444155408102228
64.599609375
0.0391002704583644
215.33203125
0.011992278021558816
538.330078125
0.010744683121207353
64.599609375
0.03474793657550931
129.19921875
0.017205921002473794
236.865234375
0.009886767546497317
64.599609375
0.0251708027122

: 